<a href="https://colab.research.google.com/github/AbeeraJadoon/Artificial-Intelligence/blob/main/lab_task_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Implement A* Algorithm for Pathfinding
# You are given a 5x5 grid where cells marked as 1 are obstacles, and 0 represents free space.
# Your task is to implement the A* algorithm in Python to find the shortest path from the topleft corner (0, 0) to the bottom-right corner (4, 4). The heuristic function should use
# Manhattan Distance. Output the grid with the path marked as P.

import heapq

def astar(grid):
  """
  Implements the A* algorithm for pathfinding.

  Args:
    grid: A 2D list representing the grid.

  Returns:
    A list of coordinates representing the shortest path, or None if no path is found.
  """
  rows, cols = len(grid), len(grid[0])
  start = (0, 0)
  end = (rows - 1, cols - 1)

  # Define the heuristic function (Manhattan Distance)
  def heuristic(node, goal):
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

  # Initialize the priority queue (using a min-heap)
  priority_queue = [(0, start)]

  # Initialize the cost dictionary (distance from the start node)
  cost = {start: 0}

  # Initialize the parent dictionary (to reconstruct the path)
  parent = {}

  while priority_queue:
    current_cost, current_node = heapq.heappop(priority_queue)

    if current_node == end:
      # Path found, reconstruct the path
      path = []
      while current_node in parent:
        path.append(current_node)
        current_node = parent[current_node]
      path.append(start)
      return path[::-1]

    neighbors = [(current_node[0] + dx, current_node[1] + dy)
                 for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]
                 if 0 <= current_node[0] + dx < rows and 0 <= current_node[1] + dy < cols and grid[current_node[0] + dx][current_node[1] + dy] == 0]

    for neighbor in neighbors:
      new_cost = cost[current_node] + 1
      if neighbor not in cost or new_cost < cost[neighbor]:
        cost[neighbor] = new_cost
        priority = new_cost + heuristic(neighbor, end)
        heapq.heappush(priority_queue, (priority, neighbor))
        parent[neighbor] = current_node

  return None  # No path found


# Example grid
grid = [
    [0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

path = astar(grid)

if path:
  for x, y in path:
    grid[x][y] = 'P'

  for row in grid:
    print(row)
else:
  print("No path found.")

['P', 'P', 'P', 'P', 'P']
[0, 1, 0, 1, 'P']
[0, 0, 0, 0, 'P']
[0, 1, 1, 1, 'P']
[0, 0, 0, 0, 'P']


In [5]:
# prompt: Solve the Water-Jug Problem using A* Algorithm
# You are given two jugs with capacities of 4 liters and 3 liters. The goal is to measure exactly
# 2 liters using these jugs. Implement the A* algorithm to solve the water-jug problem.
# The possible actions are:
# 1. Fill either jug completely.
# 2. Empty either jug.
# 3. Pour water from one jug into another until one jug is full or the other is empty.
# Use a heuristic that minimizes the absolute difference between the current amount of water
# in either jug and the target.

import heapq

def solve_water_jug_astar(jug1_cap, jug2_cap, target):
  """
  Solves the water jug problem using A* algorithm.

  Args:
    jug1_cap: Capacity of jug 1.
    jug2_cap: Capacity of jug 2.
    target: Target amount of water to measure.

  Returns:
    A list of tuples representing the sequence of states to reach the target,
    or None if no solution is found.
  """

  def heuristic(state, target):
    """Heuristic function: minimizes the absolute difference from the target."""
    return abs(state[0] - target) + abs(state[1] - target)

  def get_neighbors(state, jug1_cap, jug2_cap):
    """Generates possible next states."""
    jug1, jug2 = state
    neighbors = []

    # Fill jug1
    neighbors.append((jug1_cap, jug2))
    # Fill jug2
    neighbors.append((jug1, jug2_cap))
    # Empty jug1
    neighbors.append((0, jug2))
    # Empty jug2
    neighbors.append((jug1, 0))
    # Pour jug1 into jug2
    pour_amount = min(jug1, jug2_cap - jug2)
    neighbors.append((jug1 - pour_amount, jug2 + pour_amount))
    # Pour jug2 into jug1
    pour_amount = min(jug2, jug1_cap - jug1)
    neighbors.append((jug1 + pour_amount, jug2 - pour_amount))

    return neighbors

  start_state = (0, 0)
  priority_queue = [(heuristic(start_state, target), start_state, [])]
  visited = set()

  while priority_queue:
    _, current_state, path = heapq.heappop(priority_queue)

    if current_state[0] == target or current_state[1] == target:
      return path + [current_state]

    if current_state in visited:
      continue
    visited.add(current_state)

    for neighbor in get_neighbors(current_state, jug1_cap, jug2_cap):
      new_path = path + [current_state]
      heapq.heappush(priority_queue, (heuristic(neighbor, target), neighbor, new_path))

  return None  # No solution found

# Example usage
jug1_capacity = 4
jug2_capacity = 3
target_amount = 2

solution = solve_water_jug_astar(jug1_capacity, jug2_capacity, target_amount)

if solution:
  print("Solution found:")
  for state in solution:
    print(state)
else:
  print("No solution found.")

Solution found:
(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)


In [ ]:
# prompt: Implement Hill-Climbing algorithm to solve the 8-queen problem.

def calculate_attacks(board):
  """Calculates the number of attacks between queens on the board."""
  attacks = 0
  n = len(board)
  for i in range(n):
    for j in range(i + 1, n):
      if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
        attacks += 1
  return attacks


def hill_climbing_8queens():
  """Solves the 8-queens problem using the Hill Climbing algorithm."""
  import random
  n = 8  # Board size
  board = [random.randint(0, n - 1) for _ in range(n)]  # Random initial placement
  current_attacks = calculate_attacks(board)

  while True:
    best_neighbor = None
    min_neighbor_attacks = float('inf')

    for i in range(n):
      for j in range(n):
        new_board = list(board)
        new_board[i] = j  # Try moving queen at row i to column j
        new_attacks = calculate_attacks(new_board)

        if new_attacks < min_neighbor_attacks:
          min_neighbor_attacks = new_attacks
          best_neighbor = new_board

    if best_neighbor is None or min_neighbor_attacks >= current_attacks:
      # No better neighbor found, reached a local maximum
      if current_attacks == 0:
        return board  # Solution found
      else:
        # Restart with a new random board to potentially escape local optima
        board = [random.randint(0, n - 1) for _ in range(n)]
        current_attacks = calculate_attacks(board)
    else:
      board = best_neighbor
      current_attacks = min_neighbor_attacks


solution = hill_climbing_8queens()
if solution:
  print("Solution found:")
  for i in range(8):
    row = ['.' for _ in range(8)]
    row[solution[i]] = 'Q'
    print(''.join(row))
else:
  print("No solution found.")

Solution found:
..Q.....
.....Q..
.Q......
......Q.
Q.......
...Q....
.......Q
....Q...


In [4]:
# prompt: Write a Program to Implement the Mini-Max algorithm for a Tic-Tac-Toe game using
# python

import random

def print_board(board):
  """Prints the Tic-Tac-Toe board."""
  for row in board:
    print(" | ".join(row))
    print("-" * 9)

def check_win(board, player):
  """Checks if the player has won the game."""
  # Check rows
  for row in board:
    if all(cell == player for cell in row):
      return True

  # Check columns
  for col in range(3):
    if all(board[row][col] == player for row in range(3)):
      return True

  # Check diagonals
  if all(board[i][i] == player for i in range(3)):
    return True
  if all(board[i][2 - i] == player for i in range(3)):
    return True

  return False

def is_full(board):
  """Checks if the board is full."""
  for row in board:
    for cell in row:
      if cell == ' ':
        return False
  return True

def evaluate(board):
  """Evaluates the board for the Minimax algorithm."""
  if check_win(board, 'X'):
    return 1  # Maximizing player (X) wins
  elif check_win(board, 'O'):
    return -1  # Minimizing player (O) wins
  else:
    return 0  # Tie

def minimax(board, depth, is_maximizing):
  """Implements the Minimax algorithm with Alpha-Beta Pruning."""
  score = evaluate(board)

  if score == 1 or score == -1 or is_full(board):
    return score

  if is_maximizing:
    best_score = float('-inf')
    for row in range(3):
      for col in range(3):
        if board[row][col] == ' ':
          board[row][col] = 'X'
          score = minimax(board, depth + 1, False)
          board[row][col] = ' '  # Undo move
          best_score = max(score, best_score)
    return best_score
  else:
    best_score = float('inf')
    for row in range(3):
      for col in range(3):
        if board[row][col] == ' ':
          board[row][col] = 'O'
          score = minimax(board, depth + 1, True)
          board[row][col] = ' '  # Undo move
          best_score = min(score, best_score)
    return best_score

def find_best_move(board):
  """Finds the best move for the AI (X) using Minimax."""
  best_score = float('-inf')
  best_move = (-1, -1)

  for row in range(3):
    for col in range(3):
      if board[row][col] == ' ':
        board[row][col] = 'X'
        score = minimax(board, 0, False)
        board[row][col] = ' '
        if score > best_score:
          best_score = score
          best_move = (row, col)

  return best_move

def play_game():
  """Plays the Tic-Tac-Toe game."""
  board = [[' ' for _ in range(3)] for _ in range(3)]
  current_player = 'X'

  while True:
    print_board(board)

    if current_player == 'X':
      row, col = find_best_move(board)
      board[row][col] = 'X'
    else:
      while True:
        try:
          row = int(input("Enter row (0-2): "))
          col = int(input("Enter column (0-2): "))
          if board[row][col] == ' ':
            board[row][col] = 'O'
            break
          else:
            print("Invalid move. Try again.")
        except (ValueError, IndexError):
          print("Invalid input. Try again.")

    if check_win(board, current_player):
      print_board(board)
      print(f"Player {current_player} wins!")
      break
    elif is_full(board):
      print_board(board)
      print("It's a tie!")
      break

    current_player = 'O' if current_player == 'X' else 'X'

if __name__ == "__main__":
  play_game()

  |   |  
---------
  |   |  
---------
  |   |  
---------
X |   |  
---------
  |   |  
---------
  |   |  
---------
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Enter column (0-2): 1
Invalid input. Try again.


KeyboardInterrupt: Interrupted by user

In [6]:
# prompt: Write a Program to Implement the Mini-Max algorithm for a Tic-Tac-Toe game using
# python

import random

def print_board(board):
  """Prints the Tic-Tac-Toe board."""
  for row in board:
    print(" | ".join(row))
    print("-" * 9)

def check_win(board, player):
  """Checks if the player has won the game."""
  # Check rows
  for row in board:
    if all(cell == player for cell in row):
      return True

  # Check columns
  for col in range(3):
    if all(board[row][col] == player for row in range(3)):
      return True

  # Check diagonals
  if all(board[i][i] == player for i in range(3)):
    return True
  if all(board[i][2 - i] == player for i in range(3)):
    return True

  return False

def is_full(board):
  """Checks if the board is full."""
  for row in board:
    for cell in row:
      if cell == ' ':
        return False
  return True

def evaluate(board):
  """Evaluates the board for the Minimax algorithm."""
  if check_win(board, 'X'):
    return 1  # Maximizing player (X) wins
  elif check_win(board, 'O'):
    return -1  # Minimizing player (O) wins
  else:
    return 0  # Tie

def minimax(board, depth, is_maximizing):
  """Implements the Minimax algorithm with Alpha-Beta Pruning."""
  score = evaluate(board)

  if score == 1 or score == -1 or is_full(board):
    return score

  if is_maximizing:
    best_score = float('-inf')
    for row in range(3):
      for col in range(3):
        if board[row][col] == ' ':
          board[row][col] = 'X'
          score = minimax(board, depth + 1, False)
          board[row][col] = ' '  # Undo move
          best_score = max(score, best_score)
    return best_score
  else:
    best_score = float('inf')
    for row in range(3):
      for col in range(3):
        if board[row][col] == ' ':
          board[row][col] = 'O'
          score = minimax(board, depth + 1, True)
          board[row][col] = ' '  # Undo move
          best_score = min(score, best_score)
    return best_score

def find_best_move(board):
  """Finds the best move for the AI (X) using Minimax."""
  best_score = float('-inf')
  best_move = (-1, -1)

  for row in range(3):
    for col in range(3):
      if board[row][col] == ' ':
        board[row][col] = 'X'
        score = minimax(board, 0, False)
        board[row][col] = ' '
        if score > best_score:
          best_score = score
          best_move = (row, col)

  return best_move

def play_game():
  """Plays the Tic-Tac-Toe game."""
  board = [[' ' for _ in range(3)] for _ in range(3)]
  current_player = 'X'

  while True:
    print_board(board)

    if current_player == 'X':
      row, col = find_best_move(board)
      board[row][col] = 'X'
    else:
      while True:
        try:
          row = int(input("Enter row (0-2): "))
          col = int(input("Enter column (0-2): "))
          if board[row][col] == ' ':
            board[row][col] = 'O'
            break
          else:
            print("Invalid move. Try again.")
        except (ValueError, IndexError):
          print("Invalid input. Try again.")

    if check_win(board, current_player):
      print_board(board)
      print(f"Player {current_player} wins!")
      break
    elif is_full(board):
      print_board(board)
      print("It's a tie!")
      break

    current_player = 'O' if current_player == 'X' else 'X'

if __name__ == "__main__":
  play_game()0

  |   |  
---------
  |   |  
---------
  |   |  
---------
X |   |  
---------
  |   |  
---------
  |   |  
---------
Enter row (0-2): 1
Enter column (0-2): 2
X |   |  
---------
  |   | O
---------
  |   |  
---------
X |   | X
---------
  |   | O
---------
  |   |  
---------
Enter row (0-2): 1
Enter column (0-2): 0
X |   | X
---------
O |   | O
---------
  |   |  
---------
X | X | X
---------
O |   | O
---------
  |   |  
---------
Player X wins!
